# Ausgangscode von Aufgabe 2

In [ ]:
import cv2
import numpy as np

# Open the image file
img = cv2.imread('Gesicht.jpg')

# Set up the mouse callback function
def draw_circle(event,x,y,flags,param):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img,(x,y),3,(0,0,255),-1)    #punkt erzeugen
        points.append((x,y))    #Punkte die angelickt wurden speichern
        print(points)
        if len(points) > 2:
            cv2.polylines(img, [np.array(points)], False, (0, 255, 0), thickness=2) #image, liste aus Punkten in zweier Tupel, soll das Polygon geschlossen werden, Farbe, dicke Linie

def Cutout():
    pts = np.array(points)   #np Array aus den Punkten des Polygon erstellen
    mask = np.zeros(img.shape[:2], np.uint8) #Maske erzeugen aus den Punkten
    cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA) #Konturen zeichnen

    cutImg = cv2.bitwise_and(img, img, mask=mask)  #Bildbereich ausschneiden
    cv2.imwrite("Cut Image.png", cutImg)    #Ausgeschnittenes Bild speichern

    # Erstelle ein neues Bild mit einem leeren Alphakanal
    cutImgWithAlpha = cv2.cvtColor(cutImg, cv2.COLOR_BGR2BGRA)
    cutImgWithAlpha[:, :, 3] = mask #Maske auf das neue Bild mir Alphakanal anwenden

    # Erstelle ein neues Bild mit einem leeren Alphakanal im Hintergrund
    background = np.zeros((img.shape[0], img.shape[1], 4), dtype=np.uint8)

    # Invertiere die Maske, um das ausgeschnittene Gesicht transparent zu machen
    mask_inv = cv2.bitwise_not(mask)

    # Verwende die Maske, um den Hintergrund transparent zu machen
    background[mask_inv == 255] = [0, 0, 0, 0]

    # Füge das ausgeschnittene Gesicht zum Hintergrund hinzu
    finalImg = cv2.add(background, cutImgWithAlpha)
    cv2.imwrite("Final Image.png", finalImg)

# Neues Ausgabe Window erzeugen
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)
points = [] #Liste um Punkte des Polygones zu speichern

# Loop das die Tasten abfrägt
while True:
    cv2.imshow('image',img)  #Ausgabe des Image in einer Unendlichen Schleife
    
    if cv2.waitKey(1) == ord('q'):  #Abbruch mit der Taste q
        break

    if cv2.waitKey(1) == ord('c'):  #Ausschneiden und Speichern des Bereichs mit der Taste c 
        Cutout()

cv2.destroyAllWindows()   #Anwendung beenden

# Funktionierende Augenerkennung bei Gesicht.png

In [47]:
import cv2
import numpy as np
import math

#Berechnen des Winkels zwischen den beiden Punkten        
def CalcAngle(cords):
    print("Cords: ",cords)
    deltaX = cords[0][0] - cords[1][0]
    deltaY = cords[0][1] - cords[1][1]
    print(deltaX,deltaY)
    tan = deltaY/deltaX
    alpha = math.degrees(math.tan(tan))
    return alpha

# Laden der beiden Bilder
image1 = cv2.imread('CutFace.png')
image2 = cv2.imread('Gesicht.png')

# Erstellen der neuen Ausgabe-Matrix
img = np.zeros((max(image1.shape[0], image2.shape[0]), image1.shape[1]+image2.shape[1], 3), dtype='uint8')

# Einfügen des ersten Bildes
img[0:image1.shape[0], 0:image1.shape[1]] = image1

# Einfügen des zweiten Bildes
img[0:image2.shape[0], image1.shape[1]:image1.shape[1]+image2.shape[1]] = image2

# Überprüfen, ob das Bild erfolgreich eingelesen wurde
if img is None:
    print('Fehler: Konnte das Bild nicht einlesen')
    exit()

# Überprüfen, ob das Bild einen Alpha-Kanal hat
if len(img.shape) == 3 and img.shape[2] == 4:
    print("Das Bild hat einen Alphakanal")
    alpha = img[:, :, 3]
    img_without_alpha = img[:, :, :3]
else:
    print("Das Bild hat keinen Alphakanal")
    alpha = None
    img_without_alpha = img

# Umwandeln in Graustufenbild
gray = cv2.cvtColor(img_without_alpha, cv2.COLOR_BGR2GRAY)

# Haar-Cascade-Datei für Augen laden
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Erkennen der Augen
eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
print("Augen: ",eyes)

#Augen Positionen herausfiltern (x,y) und sortieren von augen Position links nach rechts
eyesPos = list()
for i in range(len(eyes)):
    eyesPos.append((eyes[i][0],eyes[i][1]))

print(sorted(eyesPos))

angleLeft = CalcAngle(eyesPos[:2])
angleRight = CalcAngle(eyesPos[2:])

print(angleLeft, angleRight)

# Zeichnen von Rechtecken um die erkannten Augen
for (x, y, w, h) in eyes:
    cv2.rectangle(img_without_alpha, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Zusammenführen von Alpha-Kanal und Bildoutput
# Wenn Bild Alphakanal hat
if alpha is not None:
    result = np.dstack((img_without_alpha, alpha))
# Wenn Bild keinen Alphakanal hat
else:
    result = img_without_alpha

# Anzeigen des Ergebnisses
cv2.imshow('Augen erkannt', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


Das Bild hat keinen Alphakanal
Augen:  [[258 105  48  48]
 [869 105  49  49]
 [319 109  46  46]
 [931 109  46  46]]
[(258, 105), (319, 109), (869, 105), (931, 109)]
Cords:  [(258, 105), (869, 105)]
-611 0
Cords:  [(319, 109), (931, 109)]
-612 0
-0.0 -0.0


QObject::moveToThread: Current thread (0x237f740) is not the object's thread (0x2806350).
Cannot move to target thread (0x237f740)

QObject::moveToThread: Current thread (0x237f740) is not the object's thread (0x2806350).
Cannot move to target thread (0x237f740)

QObject::moveToThread: Current thread (0x237f740) is not the object's thread (0x2806350).
Cannot move to target thread (0x237f740)

QObject::moveToThread: Current thread (0x237f740) is not the object's thread (0x2806350).
Cannot move to target thread (0x237f740)

QObject::moveToThread: Current thread (0x237f740) is not the object's thread (0x2806350).
Cannot move to target thread (0x237f740)

QObject::moveToThread: Current thread (0x237f740) is not the object's thread (0x2806350).
Cannot move to target thread (0x237f740)

QObject::moveToThread: Current thread (0x237f740) is not the object's thread (0x2806350).
Cannot move to target thread (0x237f740)

QObject::moveToThread: Current thread (0x237f740) is not the object's thread